In [1]:
import urllib.request
import os
import zipfile
from scipy.io import arff
import pandas as pd
import numpy as np
from sktime.classifiers import TSDummyClassifier


In [2]:
TRAIN_SUFFIX = '_TRAIN.arff'
TEST_SUFFIC  = '_TEST.arff'
PROXY_URL = None #'ldn2tra3:3128'

I know that getting the data from www.timeseriesclassification.com is already implemented but wanted to make sure I understand how the data should be stored in pandas and I also needed the option to download through a proxy.

In [3]:
#download

dtsName = 'GunPoint'
dtsExt = '.zip'
TSCurl = 'http://www.timeseriesclassification.com/Downloads/'
url = f'{TSCurl}{dtsName}{dtsExt}'

cache_path = f'.{os.sep}.sktime_temp_data{os.sep}'
store_loc = f'{cache_path}{dtsName}{os.sep}'
file_loc = f'{store_loc}{dtsName}{dtsExt}'
if not os.path.exists(store_loc):
    os.makedirs(store_loc)


if PROXY_URL is not None:
    print('proxy')
    proxy = urllib.request.ProxyHandler({'http': proxy_url})
    opener = urllib.request.build_opener(proxy)
    urllib.request.install_opener(opener)

if not os.path.exists(file_loc):
    urllib.request.urlretrieve(url, file_loc)  

In [4]:
#unzip
zip_ref = zipfile.ZipFile(file_loc, 'r')
zip_ref.extractall(store_loc)
zip_ref.close()

In [5]:
#import from arff
TARGET_ATTRIBUTE_NAME = 'target'
data, meta = arff.loadarff(f'{store_loc}{dtsName}{TRAIN_SUFFIX}')
df = pd.DataFrame(data)
target_ser = pd.Series(df[TARGET_ATTRIBUTE_NAME])
target_ser = target_ser.apply(int)
data_df = df.drop(TARGET_ATTRIBUTE_NAME,axis=1)

In [6]:
num_items = data_df.shape[0]
# sktime_df = pd.DataFrame({'dim1':np.empty(num_items),'target':target_ser})
sktime_df = pd.DataFrame(columns=['dim1','target'])
for i in range(num_items):
        sktime_df.at[i,'dim1'] = 1
        sktime_df.at[i,'dim1'] = data_df.iloc[i,:]
        sktime_df.at[i,'target'] = target_ser.iloc[i]

In [7]:
X = sktime_df['dim1']
y = sktime_df['target']
model = TSDummyClassifier(strategy='most')
model.fit(X, y)
preds = model.predict(X)


Preliminary design of the evaluation class

In [57]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
class Evaluate:
    """
    skeletton for the evaluation class. 
    
    The constructor can take either the fitted estimator or a results object. 
    
    If the strucutre of the results object is agreed on X, y, estimator type and other relevant properties will be obtained directly from the results object and set in the constructor.
    
    
    
    
    Parameters
    ----------
    fitted_estimator: sktime estimator
        fitted estimator
    X: pandas DataFrame
        features
    y: pandas Series
        predictors
    """
    def __init__(self, fitted_estimator, X, y):
        self._X = X
        self._y = y
        self._fitted_estimator = fitted_estimator
        self._predictions = fitted_estimator.predict(X)
        
        self._estim_type = 'Classification' #should be obtained by checking the type of the estimator object or by accessing a property
    def eval(self):
        """
        main evaluation method. Should check the type of the estimator object that was passed (classification/regression) and calculate the respective tests.  
        
        Returns
        -------
        results: dictionary
            dictionary with the statistical tests
        """
        y = np.array(self._y, np.int16)
        pred = np.array(self._predictions, np.int16)
        if self._estim_type == 'Classification':
            results = {
                'accuracy_score': accuracy_score(y_true=y, y_pred=pred),
                'f1_score': f1_score(y_true=y, y_pred=pred)
            }
        return results
    
    def check_residuals(self):
        """
        Checks the residuals of the fitted model for autocorrelation for example by performing a Ljung-Box test
        """

In [58]:
e = Evaluate(fitted_estimator=model, X=X, y=y)

In [59]:
e.eval()

c:\users\kazakovv\appdata\local\conda\conda\envs\venv_sktime\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


{'accuracy_score': 0.52, 'f1_score': 0.0}